In [0]:
# Install necessary packages -> uncomment what is currently needed

!pip install unidecode
!pip install contractions
!pip install wordsegment
!pip install -U symspellpy
!pip install emoji --upgrade
!pip install -U imbalanced-learn
!pip install bert-for-tf2
!pip install transformers
# !pip install nltk

     |████████████████████████████████| 245kB 5.4MB/s 
     |████████████████████████████████| 317kB 6.0MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81707 sha256=28f77c8000126829fad0247f2a3ebe68632cca76fc80627af221f131fa88d405
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick
     |████████████████████████████████| 4.8MB 4.7MB/s 
     |████████████████████████████████| 2.6MB 5.1MB/s 
     |████████████████████████████████| 51kB 2.5MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42175 sha256=f25071122949b304ac0e7e5fc49a18f1a1dbb159d17e7deda51bcf6b3e526872
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji
     |████████████████████████████████| 163kB 5.0MB/s 
     |████████████████████████████████| 7.0MB 22.6MB/s 
  Found existing in

In [0]:
# All imports - DO NOT CHANGE THE ORDER OF INSTRUCTIONS
!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo

import re
import os
import sys
import json

if not 'bert_repo' in sys.path:
    sys.path.insert(0, 'bert_repo')

import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
from sklearn.model_selection import train_test_split
import spacy
from bs4 import BeautifulSoup
import unidecode
import contractions
import gensim.downloader as api
import re
import wordsegment
import pkg_resources
from symspellpy.symspellpy import SymSpell, Verbosity
import emoji
from imblearn.over_sampling import SMOTE
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Model
from tensorflow import keras
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from modeling import BertModel, BertConfig
from tokenization import FullTokenizer, convert_to_unicode
from extract_features import InputExample, convert_examples_to_features
from tqdm import tqdm
#import tensorflow_addons as tfa
# import nltk
from google.colab import auth, drive
# nltk.download('punkt')

wordsegment.load()

# Load SymSpell -> package for correcting misspellings
sym_spell = SymSpell(2, 7)

dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
bigram_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_bigramdictionary_en_243_342.txt")

sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)

# get TF logger 
log = logging.getLogger('tensorflow')
log.handlers = []

Cloning into 'bert_repo'...
remote: Enumerating objects: 336, done.
remote: Total 336 (delta 0), reused 0 (delta 0), pack-reused 336
Receiving objects: 100% (336/336), 291.40 KiB | 1.52 MiB/s, done.
Resolving deltas: 100% (184/184), done.


Using TensorFlow backend.


In [0]:
#Import data
training_examples_url = 'https://raw.githubusercontent.com/piotrjaniszewski1/SemEval-2020-Task12/master/data2019/olid-training-v1.0.tsv'
training_dataset = pd.read_csv(training_examples_url, delimiter='\t')
print(training_dataset.head())
test_tweets_url = 'https://raw.githubusercontent.com/piotrjaniszewski1/SemEval-2020-Task12/master/data2019/testset-levela.tsv'
test_tweets = pd.read_csv(test_tweets_url, delimiter='\t')
print(test_tweets.head())
test_labels_url = 'https://raw.githubusercontent.com/piotrjaniszewski1/SemEval-2020-Task12/master/data2019/labels-levela.csv'
test_labels = pd.read_csv(test_labels_url, delimiter=',', header=None, names=["id", "label"])
print(test_labels.head())
test_dataset = test_tweets.set_index("id").join(test_labels.set_index("id"))
test_dataset.head()

      id                                              tweet  ... subtask_b subtask_c
0  86426  @USER She should ask a few native Americans wh...  ...       UNT       NaN
1  90194  @USER @USER Go home you’re drunk!!! @USER #MAG...  ...       TIN       IND
2  16820  Amazon is investigating Chinese employees who ...  ...       NaN       NaN
3  62688  @USER Someone should'veTaken" this piece of sh...  ...       UNT       NaN
4  43605  @USER @USER Obama wanted liberals &amp; illega...  ...       NaN       NaN

[5 rows x 5 columns]
      id                                              tweet
0  15923  #WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...
1  27014  #ConstitutionDay is revered by Conservatives, ...
2  30530  #FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...
3  13876  #Watching #Boomer getting the news that she is...
4  60133  #NoPasaran: Unity demo to oppose the far-right...
      id label
0  15923   OFF
1  27014   NOT
2  30530   NOT
3  13876   NOT
4  60133   OFF


,tweet,label
id,,
15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OFF
27014,"#ConstitutionDay is revered by Conservatives, ...",NOT
30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,NOT
13876,#Watching #Boomer getting the news that she is...,NOT
60133,#NoPasaran: Unity demo to oppose the far-right...,OFF


# **Training and validation sets**

In [0]:
seed = 13

# prepare training examples
training_examples_A = training_dataset['tweet'][training_dataset['subtask_a'].notnull()]
training_examples_B = training_dataset['tweet'][training_dataset['subtask_b'].notnull()]
training_examples_C = training_dataset['tweet'][training_dataset['subtask_c'].notnull()]

# prepare test examples and labels
test_examples_A = test_dataset['tweet'][test_dataset['label'].notnull()]
test_labels_A = (test_dataset['label'][test_dataset['label'].notnull()] == 'OFF').astype(int)

# prepare training labels
training_labels_A = (training_dataset['subtask_a'][training_dataset['subtask_a'].notnull()] == 'OFF').astype(int)
training_labels_B = (training_dataset['subtask_b'][training_dataset['subtask_b'].notnull()] == 'TIN').astype(int)
c_mapping = {'IND': 0, 'GRP': 1, 'OTH': 2}
training_labels_C = training_dataset['subtask_c'][training_dataset['subtask_c'].notnull()].replace(c_mapping)

# split training set into training and validation
training_examples_A, validation_examples_A, training_labels_A, validation_labels_A = train_test_split(
    training_examples_A, training_labels_A, test_size=0.1, stratify=training_labels_A, random_state=seed)
training_examples_B, validation_examples_B, training_labels_B, validation_labels_B = train_test_split(
    training_examples_B, training_labels_B, test_size=0.1, stratify=training_labels_B, random_state=seed)
training_examples_C, validation_examples_C, training_labels_C, validation_labels_C = train_test_split(
    training_examples_C, training_labels_C, test_size=0.1, stratify=training_labels_C, random_state=seed)

training_x = np.asarray(training_examples_A)
validation_x = np.asarray(validation_examples_A)
training_y = np.asarray(training_labels_A)
validation_y = np.asarray(validation_labels_A)
test_x = np.asarray(test_examples_A)
test_y = np.asarray(test_labels_A)

# **Preprocessing**

### Common preprocessing functions

In [0]:
# remove html tags if exist
def strip_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    stripped_text = soup.get_text(separator=' ')
    return stripped_text


# remove unnecessary whitespaces
def remove_whitespace(text):
    text = text.strip()
    return ' '.join(text.split())


# remove accented chars (e.g. caffè -> caffe)
def remove_accented_chars(text):
    text = unidecode.unidecode(text)
    return text


# remove hashes and split words (e.g. '#fortTrump' -> 'fort trump')
def split_hashtags(text):
    splitted = text.split()
    new_word_sequence = []

    for chunk in splitted:
        if chunk[0] == '#':
            chunk = chunk[1:]
            new_word_sequence.extend(wordsegment.segment(chunk))
        else:
            new_word_sequence.append(chunk)
        
    return ' '.join(tuple(new_word_sequence))


def substitute_emojis(text):
    demojized_text = emoji.demojize(text)
    return re.compile('[_:]+').sub(' ', demojized_text)


def preprocess_common(text):
    text = strip_html_tags(text)
    text = contractions.fix(text)
    text = split_hashtags(text)
    text = substitute_emojis(text)
    text = remove_whitespace(text)
    text = remove_accented_chars(text)
    return text.lower()

In [0]:
# Remove redundant @user tokens
def remove_redundant_users(example):
    user_count = 0
    new_example = example[:]
    for i, token in reversed(list(enumerate(example))):
        if token == '@user':
            user_count += 1
        if user_count > 3:
            new_example.pop(i)
    else:
        user_count = 0

    return new_example

### Spacy preprocessing

In [0]:
# Try leaving '?' and '!' as far as punctuation is concerned

nlp = spacy.load('en_core_web_sm')

# exclude negation words from spacy stopwords list
deselect_stop_words = ['no', 'not', 'noone', 'none', 'lacks', 'lack', 'nor', 'never', 'neighter', 'hardly', 'nobody', 'nothing', 'lacking', 'nowhere']
for w in deselect_stop_words:
    nlp.vocab[w].is_stop = False

def preprocess_spacy(text):
    doc = nlp(text)

    clean_text = []
    
    for token in doc:
        flag = True
        edit = token.text

        # remove punctuations
        if token.pos_ == 'PUNCT' and flag == True and token.text != '@user': 
            flag = False
       
        # remove special characters
        if token.pos_ == 'SYM' and flag == True: 
            flag = False
        
        # remove numbers
        if (token.pos_ == 'NUM' or token.text.isnumeric()) and flag == True:
            flag = False

        # correct misspelings
        if flag == True:
            suggestions = sym_spell.lookup(edit, Verbosity.TOP, 2)
            if len(suggestions) > 0:
                edit = suggestions[0].term

        # remove stop words
        if token.is_stop and token.pos_ != 'NUM': 
            flag = False

        # convert tokens to base form
        elif token.lemma_ != '-PRON-' and flag == True:
            edit = token.lemma_

        # append tokens edited and not removed to list 
        if edit != '' and flag == True:
            clean_text.append(edit)        
    
    return clean_text

Preprocessing execution

In [0]:
# normalization -> papers, complicated solutions, replace abbreviations with full names (e.g. MAGA)
# check removing less stop words (some may have some significance)

cleaned_x = [preprocess_spacy(example) for example in training_x[0:30]]
reduced_users_x = [remove_redundant_users(example) for example in cleaned_x]
print(reduced_users_x[0:30])
print(training_x[0:30])

## Helper functions

In [0]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced',
                                            np.unique(training_y),
                                            training_y)
class_weights /= max(class_weights)

In [0]:
def weighted_loss(actual, predicted):
    bce = tf.keras.losses.BinaryCrossentropy()
    loss = bce(actual, predicted)
   
    return tf.keras.backend.mean(loss * class_weights)

# **BERT**

### BERT Preprocessing - single example


In [0]:
def remove_punctation(tokens):
    punctuation = [char for char in string.punctuation]
    return list(filter(lambda token: token not in punctuation, tokens))


def preprocess_bert_before_tokenization(text):
    preprocessed_text = preprocess_common(text)
    spacy_x = preprocess_spacy(preprocessed_text)
    cleaned_x = remove_redundant_users(spacy_x)
    return ' '.join(cleaned_x)


def preprocess_bert_after_tokenization(tokenized_x):
    # update input_ids, input_mask, input_type_ids
    return remove_punctation(cleaned_x) # remove i.a. hyphens remainings from words like 'de-platforming'

### Getting the pre-trained model

In [0]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

--2019-12-27 20:24:41--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.117.128, 2607:f8b0:400e:c02::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.117.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   119MB/s    in 3.3s    

2019-12-27 20:24:45 (119 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


### Building a tf.Module

In [0]:
def build_module_fn(config_path, vocab_path, do_lower_case=True):

    def bert_module_fn(is_training):
        """Spec function for a token embedding module."""

        input_ids = tf.placeholder(shape=[None, None], dtype=tf.int32, name="input_ids")
        input_mask = tf.placeholder(shape=[None, None], dtype=tf.int32, name="input_mask")
        token_type = tf.placeholder(shape=[None, None], dtype=tf.int32, name="segment_ids")

        config = BertConfig.from_json_file(config_path)
        model = BertModel(config=config, is_training=is_training,
                          input_ids=input_ids, input_mask=input_mask, token_type_ids=token_type)
          
        seq_output = model.all_encoder_layers[-1]
        pool_output = model.get_pooled_output()

        config_file = tf.constant(value=config_path, dtype=tf.string, name="config_file")
        vocab_file = tf.constant(value=vocab_path, dtype=tf.string, name="vocab_file")
        lower_case = tf.constant(do_lower_case)

        tf.add_to_collection(tf.GraphKeys.ASSET_FILEPATHS, config_file)
        tf.add_to_collection(tf.GraphKeys.ASSET_FILEPATHS, vocab_file)
        
        input_map = {"input_ids": input_ids,
                     "input_mask": input_mask,
                     "segment_ids": token_type}
        
        output_map = {"pooled_output": pool_output,
                      "sequence_output": seq_output}

        output_info_map = {"vocab_file": vocab_file,
                           "do_lower_case": lower_case}
                
        hub.add_signature(name="tokens", inputs=input_map, outputs=output_map)
        hub.add_signature(name="tokenization_info", inputs={}, outputs=output_info_map)

    return bert_module_fn

### Exporting the module

In [0]:
MODEL_DIR = "uncased_L-12_H-768_A-12" #@param {type:"string"} ['uncased_L-12_H-768_A-12']

config_path = "/content/{}/bert_config.json".format(MODEL_DIR)
vocab_path = "/content/{}/vocab.txt".format(MODEL_DIR)

tags_and_args = []
for is_training in (True, False):
  tags = set()
  if is_training:
    tags.add("train")
  tags_and_args.append((tags, dict(is_training=is_training)))

module_fn = build_module_fn(config_path, vocab_path)
spec = hub.create_module_spec(module_fn, tags_and_args=tags_and_args)
spec.export("bert-module", 
            checkpoint_path="/content/{}/bert_model.ckpt".format(MODEL_DIR))


From bert_repo/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

From bert_repo/modeling.py:171: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

From bert_repo/modeling.py:409: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
From bert_repo/modeling.py:490: The name tf.assert_less_equal is deprecated. Please use tf.compat.v1.assert_less_equal instead.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-co

### Building the text preprocessing pipeline

In [0]:
def read_examples(str_list):
    """Read a list of `InputExample`s from a list of strings."""
    unique_id = 0
    for s in str_list:
        line = convert_to_unicode(s)
        if not line:
            continue
        
        text_a = line.strip()
        yield InputExample(unique_id=unique_id, text_a=text_a, text_b=None)
        unique_id += 1

In [0]:
def features_to_arrays(features):

    all_input_ids = []
    all_input_mask = []
    all_segment_ids = []

    for feature in features:
        all_input_ids.append(feature.input_ids)
        all_input_mask.append(feature.input_mask)
        all_segment_ids.append(feature.input_type_ids)

    return (np.array(all_input_ids, dtype='int32'), 
            np.array(all_input_mask, dtype='int32'), 
            np.array(all_segment_ids, dtype='int32'))

In [0]:
def build_preprocessor(voc_path, seq_len, lower=True):
    tokenizer = FullTokenizer(vocab_file=voc_path, do_lower_case=lower)

    def strings_to_arrays(sents):

        sents = np.atleast_1d(sents).reshape((-1,))

        examples = []
        for example in read_examples(sents):
            example.text_a = preprocess_bert_before_tokenization(example.text_a)
            examples.append(example)

        features = convert_examples_to_features(examples, seq_len, tokenizer)
        arrays = features_to_arrays(features)
        
        return arrays

    return strings_to_arrays

### Implementing a BERT Keras layer

In [0]:
class BertLayer(tf.keras.layers.Layer):
    def __init__(self, bert_path, seq_len=64, n_tune_layers=3, 
                 pooling="cls", do_preprocessing=True, verbose=False,
                 tune_embeddings=False, trainable=True, **kwargs):

        self.trainable = trainable
        self.n_tune_layers = n_tune_layers
        self.tune_embeddings = tune_embeddings
        self.do_preprocessing = do_preprocessing

        self.verbose = verbose
        self.seq_len = seq_len
        self.pooling = pooling
        self.bert_path = bert_path

        self.var_per_encoder = 16
        if self.pooling not in ["cls", "mean", None]:
            raise NameError(
                f"Undefined pooling type (must be either 'cls', 'mean', or None, but is {self.pooling}"
            )

        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):

        self.bert = hub.Module(self.build_abspath(self.bert_path), 
                               trainable=self.trainable, name=f"{self.name}_module")

        trainable_layers = []
        if self.tune_embeddings:
            trainable_layers.append("embeddings")

        if self.pooling == "cls":
            trainable_layers.append("pooler")

        if self.n_tune_layers > 0:
            encoder_var_names = [var.name for var in self.bert.variables if 'encoder' in var.name]
            n_encoder_layers = int(len(encoder_var_names) / self.var_per_encoder)
            for i in range(self.n_tune_layers):
                trainable_layers.append(f"encoder/layer_{str(n_encoder_layers - 1 - i)}/")
        
        # Add module variables to layer's trainable weights
        for var in self.bert.variables:
            if any([l in var.name for l in trainable_layers]):
                self._trainable_weights.append(var)
            else:
                self._non_trainable_weights.append(var)

        if self.verbose:
            print("*** TRAINABLE VARS *** ")
            for var in self._trainable_weights:
                print(var)

        self.build_preprocessor()
        self.initialize_module()

        super(BertLayer, self).build(input_shape)

    def build_abspath(self, path):
        if path.startswith("https://") or path.startswith("gs://"):
          return path
        else:
          return os.path.abspath(path)

    def build_preprocessor(self):
        sess = tf.keras.backend.get_session()
        tokenization_info = self.bert(signature="tokenization_info", as_dict=True)
        vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                              tokenization_info["do_lower_case"]])
        self.preprocessor = build_preprocessor(vocab_file, self.seq_len, do_lower_case)

    def initialize_module(self):
        sess = tf.keras.backend.get_session()
        
        vars_initialized = sess.run([tf.is_variable_initialized(var) 
                                     for var in self.bert.variables])

        uninitialized = []
        for var, is_initialized in zip(self.bert.variables, vars_initialized):
            if not is_initialized:
                uninitialized.append(var)

        if len(uninitialized):
            sess.run(tf.variables_initializer(uninitialized))

    def call(self, input):

        if self.do_preprocessing:
          input = tf.numpy_function(self.preprocessor, 
                                    [input], [tf.int32, tf.int32, tf.int32], 
                                    name='preprocessor')
          for feature in input:
            feature.set_shape((None, self.seq_len))
        
        input_ids, input_mask, segment_ids = input
        
        bert_inputs = dict(
            input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids
        )
        output = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)
        
        if self.pooling == "cls":
            pooled = output["pooled_output"]
        else:
            result = output["sequence_output"]
            
            input_mask = tf.cast(input_mask, tf.float32)
            mul_mask = lambda x, m: x * tf.expand_dims(m, axis=-1)
            masked_reduce_mean = lambda x, m: tf.reduce_sum(mul_mask(x, m), axis=1) / (
                    tf.reduce_sum(m, axis=1, keepdims=True) + 1e-10)
            
            if self.pooling == "mean":
              pooled = masked_reduce_mean(result, input_mask)
            else:
              pooled = mul_mask(result, input_mask)

        return pooled

    def get_config(self):
        config_dict = {
            "bert_path": self.bert_path, 
            "seq_len": self.seq_len,
            "pooling": self.pooling,
            "n_tune_layers": self.n_tune_layers,
            "tune_embeddings": self.tune_embeddings,
            "do_preprocessing": self.do_preprocessing,
            "verbose": self.verbose
        }
        super(BertLayer, self).get_config()
        return config_dict

### Classification

In [0]:
inp = tf.keras.Input(shape=(1,), dtype=tf.string)
encoder = BertLayer(bert_path="./bert-module/", seq_len=256, tune_embeddings=False,
                    pooling='cls', n_tune_layers=2, verbose=False)

layer = tf.keras.layers.Dense(64, activation='relu')(encoder(inp))
pred = tf.keras.layers.Dense(1, activation='sigmoid')(layer)
model = tf.keras.models.Model(inputs=[inp], outputs=[pred])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [0]:
model.summary()

model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5, ),
      loss=weighted_loss,
      metrics=[f1_m])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
bert_layer (BertLayer)       (None, 768)               109482240 
_________________________________________________________________
dense (Dense)                (None, 64)                49216     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 109,531,521
Trainable params: 14,815,617
Non-trainable params: 94,715,904
_________________________________________________________________


### Training

In [0]:
import logging
logging.getLogger("tensorflow").setLevel(logging.WARNING)

In [0]:
saver = keras.callbacks.ModelCheckpoint("bert_H2_S256_B128.hdf5")

history = model.fit(training_x, training_y, validation_data=[validation_x, validation_y], batch_size=128, epochs=3, callbacks=[saver])

Train on 11916 samples, validate on 1324 samples
Epoch 1/3


From bert_repo/extract_features.py:283: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.



11916/11916 [==============================] - 304s 25ms/sample - loss: 0.4373 - f1_m: 0.1981 - val_loss: 0.3761 - val_f1_m: 0.5533
Epoch 2/3
11916/11916 [==============================] - 295s 25ms/sample - loss: 0.3700 - f1_m: 0.5817 - val_loss: 0.3600 - val_f1_m: 0.5802
Epoch 3/3
11916/11916 [==============================] - 292s 25ms/sample - loss: 0.3498 - f1_m: 0.6142 - val_loss: 0.3541 - val_f1_m: 0.6086


In [0]:
model.evaluate(test_x, test_y)

860/860 [==============================] - 20s 23ms/sample - loss: 0.3079 - f1_m: 0.5873


[0.3079082017721132, 0.587346]

### Loss function graph

In [0]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Saving and restoring

In [0]:
print(training_y[:10])
model.predict(training_x[:10])

[1 1 0 0 1 0 0 0 1 0]


array([[0.67689705],
       [0.824897  ],
       [0.08231309],
       [0.07174948],
       [0.8153479 ],
       [0.1083402 ],
       [0.37947318],
       [0.52877355],
       [0.7440483 ],
       [0.0785928 ]], dtype=float32)

In [0]:
import json
json.dump(model.to_json(), open("bert_H2_S256_B64.json", "w"))

In [0]:
model = tf.keras.models.model_from_json(json.load(open("bert_H2_S256_B64.json")), 
                                        custom_objects={"BertLayer": BertLayer})

model.load_weights("bert_H2_S256_B64.hdf5")

In [0]:
model.predict(training_x[:10])

### Freeze model

In [0]:
from tensorflow.python.framework.graph_util import convert_variables_to_constants
from tensorflow.python.tools.optimize_for_inference_lib import optimize_for_inference

def freeze_keras_model(model, export_path=None, clear_devices=True):
    """
    Freezes a Keras model into a pruned computation graph.

    @param model The Keras model to be freezed.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    
    sess = tf.keras.backend.get_session()
    graph = sess.graph
    
    with graph.as_default():

        input_tensors = model.inputs
        output_tensors = model.outputs
        dtypes = [t.dtype.as_datatype_enum for t in input_tensors]
        input_ops = [t.name.rsplit(":", maxsplit=1)[0] for t in input_tensors]
        output_ops = [t.name.rsplit(":", maxsplit=1)[0] for t in output_tensors]
        
        tmp_g = graph.as_graph_def()
        if clear_devices:
            for node in tmp_g.node:
                node.device = ""
        
        tmp_g = optimize_for_inference(
            tmp_g, input_ops, output_ops, dtypes, False)
        
        tmp_g = convert_variables_to_constants(sess, tmp_g, output_ops)
        
        if export_path is not None:
            with tf.gfile.GFile(export_path, "wb") as f:
                f.write(tmp_g.SerializeToString())
        
        return tmp_g

In [0]:
frozen_graph = freeze_keras_model(model, export_path="frozen_graph_256_64.pb")

In [0]:
!git clone https://github.com/gaphex/bert_experimental/

import tensorflow as tf
import numpy as np
import sys

sys.path.insert(0, "/content/bert_experimental")

from bert_experimental.finetuning.text_preprocessing import build_preprocessor
from bert_experimental.finetuning.graph_ops import load_graph

fatal: destination path 'bert_experimental' already exists and is not an empty directory.


In [0]:
restored_graph = load_graph("frozen_graph_256_64.pb")

In [0]:
graph_ops = restored_graph.get_operations()
input_op, output_op = graph_ops[0].name, graph_ops[-1].name
print(input_op, output_op)

import/input_2_1 import/dense_1_2/Sigmoid


In [0]:
x = restored_graph.get_tensor_by_name(input_op + ':0')
y = restored_graph.get_tensor_by_name(output_op + ':0')

In [0]:
preprocessor = build_preprocessor("./uncased_L-12_H-768_A-12/vocab.txt", 64)
py_func = tf.numpy_function(preprocessor, [x], [tf.int32, tf.int32, tf.int32], name='preprocessor')

In [0]:
py_func = tf.numpy_function(preprocessor, [x], [tf.int32, tf.int32, tf.int32])

In [0]:
sess = tf.Session(graph=restored_graph)

In [0]:
y_out = sess.run(y, feed_dict={
        x: training_x[:10].reshape((-1,1))
    })

y_out

# LSTM model

## Preparing data

In [0]:
from google.colab import drive
drive.mount('/content/drive')
PATH_TO_DIR = '/content/drive/My Drive/STUDIA/SEM 9/pracownia/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def full_preprocessing(example):
    basic_preprocessed = preprocess_common(example)
    cleaned_example = preprocess_spacy(basic_preprocessed)
    return remove_redundant_users(cleaned_example)

In [0]:
preprocessed_train = [full_preprocessing(example) for example in training_x]
preprocessed_valid = [full_preprocessing(example) for example in validation_x]
preprocessed_test = [full_preprocessing(example) for example in test_x]

In [0]:
from gensim.models import Word2Vec
word2vec = Word2Vec.load(PATH_TO_DIR + "corrected_word2vec.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
sim_words = word2vec.wv.most_similar('maga')

for i in sim_words[:10]:
    print(i)

('kag', 0.9932580590248108)
('wwg1wga', 0.9898135662078857)
('patriot', 0.9889000654220581)
('drain', 0.9829832315444946)
('6th', 0.9824297428131104)
('deep', 0.9804140329360962)
('state', 0.9798689484596252)
('kag2018', 0.9782142043113708)
('q', 0.9760551452636719)
('united', 0.9707470536231995)


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
def word2idx(word):
    try:
        return word2vec.wv.vocab[word].index
    except KeyError:
        return 0
        
def idx2word(token):
    return word2vec.wv.index2word[token]

In [0]:
def word2idx_set(X):
    new_X = []

    for line in X:
        new_line = []
        for w in line:
            new_line.append(word2idx(w))
        new_X.append(new_line)
    
    return new_X

In [0]:
max_lenght = 79
w2v_weights = word2vec.wv.vectors
vocab_size, embedding_size = w2v_weights.shape

print("Vocabulary Size: {} - Embedding Dim: {}".format(vocab_size, embedding_size))

Vocabulary Size: 7457 - Embedding Dim: 100


In [0]:
from keras.preprocessing.sequence import pad_sequences

X_train = np.asarray(word2idx_set(preprocessed_train))
X_train = pad_sequences(X_train, maxlen=max_lenght, padding='pre', value=0)

X_valid = np.asarray(word2idx_set(preprocessed_valid))
X_valid = pad_sequences(X_valid, maxlen=max_lenght, padding='pre', value=0)

X_test = np.asarray(word2idx_set(preprocessed_test))
X_test = pad_sequences(X_test, maxlen=max_lenght, padding='pre', value=0)

X_train.shape, X_valid.shape, X_test.shape

In [0]:
X_test[0]    # exemplary sequence

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,  178,  406, 4419, 1461,  678,    0, 3758,
       1166,   33,   51,    7,  481, 3280, 4582, 1625, 4616,  141, 5864,
       1036,  297,  536,  192,  594, 5552, 1052,  592,  260,  178,  145,
          0,    2], dtype=int32)

## Training model

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.initializers import Constant

In [0]:
import os

embeddings_index = {}
with open(os.path.join(PATH_TO_DIR, 'glove.twitter.27B.100d.txt')) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

# prepare embedding matrix
EMBEDDING_DIM = 100
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))

for word, vec in word2vec.wv.vocab.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[vec.index] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer_pretrained = Embedding(vocab_size,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=max_lenght,
                            trainable=False,
                            mask_zero=True)

In [0]:
lstm_model = Sequential()

# Keras Embedding layer with Word2Vec weights initialization
lstm_model.add(embedding_layer_pretrained)

lstm_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
lstm_model.add(Dense(64, activation='relu'))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.summary()

lstm_model.compile(optimizer='adam', loss=weighted_loss, metrics=[precision_m, recall_m, f1_m])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 79, 100)           745700    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                6464      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 832,629
Trainable params: 86,929
Non-trainable params: 745,700
_________________________________________________________________


In [0]:
def get_model_checkpoint(name):
    return ModelCheckpoint(PATH_TO_DIR + 'models/' + name,
                     monitor='val_loss',
                     mode='min',
                     verbose=1,
                     save_best_only=True)

In [0]:
## using pretrained embeddings trainable!
model_name = 'weighted_drop_glove_more_dense'

history = lstm_model.fit(X_train, training_y, epochs=10, batch_size=64,
                    validation_data=(X_valid, validation_y), 
                    callbacks=[get_model_checkpoint('best_' + model_name + '.h5')])

lstm_model.save_weights(PATH_TO_DIR + 'models/' + model_name + '.h5')

Train on 11916 samples, validate on 1324 samples
Epoch 1/10
11916/11916 [==============================] - 30s 3ms/step - loss: 0.4180 - precision_m: 0.5955 - recall_m: 0.3826 - f1_m: 0.4446 - val_loss: 0.3719 - val_precision_m: 0.7196 - val_recall_m: 0.5074 - val_f1_m: 0.5842

Epoch 00001: val_loss improved from inf to 0.37186, saving model to /content/drive/My Drive/STUDIA/SEM 9/pracownia/models/best_weighted_drop_glove_more_dense.h5
Epoch 2/10
11916/11916 [==============================] - 28s 2ms/step - loss: 0.3705 - precision_m: 0.7020 - recall_m: 0.4955 - f1_m: 0.5696 - val_loss: 0.3620 - val_precision_m: 0.7481 - val_recall_m: 0.5092 - val_f1_m: 0.5939

Epoch 00002: val_loss improved from 0.37186 to 0.36202, saving model to /content/drive/My Drive/STUDIA/SEM 9/pracownia/models/best_weighted_drop_glove_more_dense.h5
Epoch 3/10
11916/11916 [==============================] - 28s 2ms/step - loss: 0.3570 - precision_m: 0.7298 - recall_m: 0.5100 - f1_m: 0.5898 - val_loss: 0.3537 - va

In [0]:
lstm_model.evaluate(X_test, test_y)

860/860 [==============================] - 2s 2ms/step


[0.2912720369738202,
 0.7705164645993432,
 0.49533670281254966,
 0.583955024563989]

## Loading model

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.initializers import Constant

In [0]:
lstm_model = Sequential()

# Keras Embedding layer with Word2Vec weights initialization
lstm_model.add(Embedding(vocab_size,
                          embedding_size,
                          input_length=max_lenght,
                          trainable=False,
                          mask_zero=True))

lstm_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
lstm_model.add(Dense(64, activation='relu'))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.summary()

lstm_model.compile(optimizer='adam', loss=weighted_loss, metrics=[precision_m, recall_m, f1_m])

In [0]:
lstm_model.load_weights(PATH_TO_DIR + 'models/best_weighted_drop_glove_more_dense.h5')

# BERT and LSTM ensemble

In [0]:
from tensorflow.keras.layers import Concatenate

In [0]:
for layer in model.layers:
    layer.trainable = False

for layer in lstm_model.layers:
    layer.trainable = False

In [0]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
bert_layer (BertLayer)       (None, 768)               109482240 
_________________________________________________________________
dense (Dense)                (None, 64)                49216     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 109,531,521
Trainable params: 14,815,617
Non-trainable params: 94,715,904
_________________________________________________________________


In [0]:
lstm_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 79, 100)           745700    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                6464      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 832,629
Trainable params: 86,929
Non-trainable params: 745,700
_________________________________________________________________


In [0]:
concatenated = Concatenate()([model.layers[-2].output, lstm_model.layers[-2].output])
output = Dense(1, activation='sigmoid')(concatenated)
new_model = Model(inputs=[model.layers[0].input, lstm_model.layers[0].input], outputs=output)

In [0]:
new_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
embedding_input (InputLayer)    [(None, 79)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 79, 100)      745700      embedding_input[0][0]            
__________________________________________________________________________________________________
bert_layer (BertLayer)          (None, 768)          109482240   input_1[0][0]                    
____________________________________________________________________________________________

In [0]:
new_model.compile(loss=weighted_loss, optimizer='adam', metrics=[f1_m])

In [0]:
history = new_model.fit([training_x, X_train], training_y, epochs=2, batch_size=64,
                    validation_data=([validation_x, X_valid], validation_y))

new_model.save_weights(PATH_TO_DIR + 'models/full_model.h5')

Train on 11916 samples, validate on 1324 samples
Epoch 1/2
11916/11916 [==============================] - 274s 23ms/sample - loss: 0.3642 - f1_m: 0.5915 - val_loss: 0.3408 - val_f1_m: 0.6253
Epoch 2/2
11916/11916 [==============================] - 266s 22ms/sample - loss: 0.3193 - f1_m: 0.6508 - val_loss: 0.3390 - val_f1_m: 0.6362


In [0]:
new_model.evaluate([test_x, X_test], test_y)

860/860 [==============================] - 24s 27ms/sample - loss: 0.2831 - f1_m: 0.6459


[0.28307747882465983, 0.6458599]